In [ ]:
import bz2
import os

from urllib.request import urlopen

def download_landmarks(dst_file):
    url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
    decompressor = bz2.BZ2Decompressor()
    
    with urlopen(url) as src, open(dst_file, 'wb') as dst:
        data = src.read(1024)
        while len(data) > 0:
            dst.write(decompressor.decompress(data))
            data = src.read(1024)

dst_dir = 'models'
dst_file = os.path.join(dst_dir, 'landmarks.dat')

if not os.path.exists(dst_file):
    os.makedirs(dst_dir)
    download_landmarks(dst_file)

In [ ]:
from model import create_model
nn4_small2 = create_model()

In [ ]:
nn4_small2.summary()

In [ ]:

from keras import backend as K
from keras.models import Model
from keras.layers import Input,Layer

in_a = Input(shape=(96,96,3))
in_p = Input(shape=(96,96,3))
in_n = Input(shape=(96,96,3))

emb_a = nn4_small2(in_a)
emb_p = nn4_small2(in_p)
emb_n = nn4_small2(in_n)

class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=0.2, name='triplet_loss_layer')([emb_a, emb_p, emb_n])

# Model that can be trained with anchor, positive negative images
nn4_small2_train = Model([in_a, in_p, in_n], triplet_loss_layer)

In [ ]:
from data import triplet_generator

# triplet_generator() creates a generator that continuously returns 
# ([a_batch, p_batch, n_batch], None) tuples where a_batch, p_batch 
# and n_batch are batches of anchor, positive and negative RGB images 
# each having a shape of (batch_size, 96, 96, 3).
generator = triplet_generator() 

nn4_small2_train.compile(loss=None, optimizer='adam')
nn4_small2_train.fit_generator(generator, epochs=10, steps_per_epoch=100)

# Please note that the current implementation of the generator only generates 
# random image data. The main goal of this code snippet is to demonstrate 
# the general setup for model training. In the following, we will anyway 
# use a pre-trained model so we don't need a generator here that operates 
# on real training data. I'll maybe provide a fully functional generator
# later.

In [ ]:
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [ ]:
import numpy as np
import os.path

class IdentityMetadata():
    def __init__(self, base, file):
        # dataset base directory
        self.base = base
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.file) 
    
def load_metadata(path):
    metadata = []
    for i in os.listdir(path):
        ext = os.path.splitext(i)[1]
        metadata.append(IdentityMetadata(path, i))
    return np.array(metadata)

metadata_test_faces = load_metadata('test_faces')
metadata_test_gan_sample = load_metadata('test_gan_sample')
metadata_test_gan_sample_1 = load_metadata('test_gan_sample_1')
metadata_test_vae_sample = load_metadata('test_vae_sample')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from align import AlignDlib

%matplotlib inline

def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    return img[...,::-1]

# Initialize the OpenFace face alignment utility
alignment = AlignDlib('models/landmarks.dat')

# Load an image of Jacques Chirac
jc_orig = load_image(metadata_test_faces[100].image_path())

# Detect face and return bounding box
bb = alignment.getLargestFaceBoundingBox(jc_orig)

# Transform image using specified face landmark indices and crop image to 96x96
jc_aligned = alignment.align(96, jc_orig, bb, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

# Show original image
plt.subplot(131)
plt.imshow(jc_orig)

# Show original image with bounding box
plt.subplot(132)
plt.imshow(jc_orig)
plt.gca().add_patch(patches.Rectangle((bb.left(), bb.top()), bb.width(), bb.height(), fill=False, color='red'))

# Show aligned image
plt.subplot(133)
plt.imshow(jc_aligned);

In [ ]:
def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [ ]:
embedded_test_faces = np.zeros((metadata_test_faces.shape[0], 128))
embedded_test_gan_sample = np.zeros((metadata_test_gan_sample.shape[0], 128))
embedded_test_gan_sample_1 = np.zeros((metadata_test_gan_sample_1.shape[0], 128))
embedded_test_vae_sample = np.zeros((metadata_test_vae_sample.shape[0], 128))
corrupt = []
gan_path = 'test_gan_sample'
gan_1_path = 'test_gan_sample_1'
vae_path = 'test_vae_sample'
test_path = 'test_faces'
for i, m in enumerate(metadata_test_vae_sample):
    m1=str(m)
    file = m1.split('/')
    test_img = load_image(os.path.join(test_path,file[1]))
    gan_img = load_image(os.path.join(gan_path,file[1]))
    gan_1_img = load_image(os.path.join(gan_1_path,file[1]))
    vae_img = load_image(os.path.join(vae_path,file[1]))
    test_img = align_image(test_img)
    gan_img = align_image(gan_img)
    gan_1_img = align_image(gan_1_img)
    vae_img = align_image(vae_img)
    if test_img is not None and gan_img is not None and gan_1_img is not None and vae_img is not None:
        # scale RGB values to interval [0,1]
        test_img = (test_img / 255.).astype(np.float32)
        gan_img = (gan_img / 255.).astype(np.float32)
        gan_1_img = (gan_1_img / 255.).astype(np.float32)
        vae_img = (vae_img / 255.).astype(np.float32)
    # obtain embedding vector for image
    try:
        embedded_test_faces[i] = nn4_small2_pretrained.predict(np.expand_dims(test_img, axis=0))[0]
        embedded_test_gan_sample[i] = nn4_small2_pretrained.predict(np.expand_dims(gan_img, axis=0))[0]
        embedded_test_gan_sample_1[i] = nn4_small2_pretrained.predict(np.expand_dims(gan_1_img, axis=0))[0]
        embedded_test_vae_sample[i] = nn4_small2_pretrained.predict(np.expand_dims(vae_img, axis=0))[0]
    except Exception as e:
        corrupt.append(i)
        


In [ ]:
embedded_test_faces = np.delete(embedded_test_faces,corrupt,0)
embedded_test_gan_sample = np.delete(embedded_test_gan_sample,corrupt,0)
embedded_test_gan_sample_1 = np.delete(embedded_test_gan_sample_1,corrupt,0)
embedded_test_vae_sample = np.delete(embedded_test_vae_sample,corrupt,0)

In [ ]:
metadata_test_faces = np.delete(metadata_test_faces,corrupt,0)
metadata_test_gan_sample = np.delete(metadata_test_gan_sample,corrupt,0)
metadata_test_gan_sample_1 = np.delete(metadata_test_gan_sample_1,corrupt,0)
metadata_test_vae_sample = np.delete(metadata_test_vae_sample,corrupt,0)

In [ ]:
def  distance (emb1, emb2):
    return np.sum(np.square(emb1 - emb2))

def show_pair(idx1, idx2):
    plt.figure(figsize=(8,3))
    plt.suptitle(f'Distance = {distance(embedded_test_faces[idx1], embedded_test_vae_sample[idx2]):.2f}')
    plt.subplot(121)
    plt.imshow(load_image(metadata_test_faces[idx1].image_path()))
    plt.subplot(122)
    plt.imshow(load_image(metadata_test_vae_sample[idx2].image_path()));    

show_pair(9, 9)

In [ ]:
gan_distances = []
gan_1_distances = []
vae_distances = []

num = 400

for i in range(num):
    gan_distances.append(distance(embedded_test_faces[i], embedded_test_gan_sample[i]))
    gan_1_distances.append(distance(embedded_test_faces[i], embedded_test_gan_sample_1[i]))
    vae_distances.append(distance(embedded_test_faces[i], embedded_test_vae_sample[i]))                

In [ ]:
correct_gan=0
correct_gan_1=0
correct_vae=0

for i in range(400):
    if(gan_distances[i]<=0.77):
        correct_gan+=1
    if(gan_1_distances[i]<=0.77):
        correct_gan_1+=1
    if(vae_distances[i]<=0.77):
        correct_vae+=1    
        
print("Accuracy for CGAN",(correct_gan/400)*100,"%")
print("Accuracy for RCGAN",(correct_gan_1/400)*100,"%")
print("Accuracy for VAE",(correct_vae/400)*100,"%")